In [4]:
### CONGREGATING DATA ###
import os
all_files = ""
directory = "mini_LDR"

#for each file in directory, append its contents to all_files string
for filename in os.listdir(directory):
    if filename.endswith(".ldr"):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            all_files += file.read()

In [8]:
### PRE-TOKENISATION USING GPT4 ###
import regex as regx
gpt4_pattern = regx.compile(r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""")
#https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py

all_sub_units = regx.findall(gpt4_pattern, all_files)

In [ ]:
### CONVERT EACH CHAR TO UTF-8 RAW BYTES AND INITALISE TOKEN SET ###

#for each sub-unit 
    #convert to raw bytes
    #convert those raw bytes to ints



